In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Finding Feasible Solutions

Constrained, gradient-based optimization tools like SLSQP, IPOPT, and SNOPT can benefit from starting from a feasible design point,
but finding a feasible design with dozens or hundreds of design variables can be a challenge in itself.

Domain-specific methods exist for finding a feasible starting point, such as using a suboptimal explicit simulation to jump-start an implicit trajectory optimization, but in a multidisciplinary environment a more general approach is needed.

OpenMDAO's `Problem.find_feasible` method aims to find a feasible point if one exists, or otherwise minimize the violation of infeasible constraints in the design. 

This approach poses the constraint violations as residuals to be minimized via a least-squares approach.
Under the hood, it utilizes [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) to perform this minimization. 

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.find_feasible
        :noindex:
```

Consider the following optimization problem. It contains two constraints which are in conflict with each other.

\begin{align}
    \mathrm{Minimize}: \\
    y = x_1^2 - x_2^2 \\
    \mathrm{Subject\,to:} \\
    g_1 = 2 x_1 - 5 \\
    g_2 = -2 x_1 - 5 \\
    g_3 = x_2 - 5
\end{align}


The OpenMDAO problem can be formulated as follows:

In [ ]:
import numpy as np
import openmdao.api as om

prob = om.Problem()

c1 = om.ExecComp()
c1.add_expr('y = x1**2 - x2**2', y={'copy_shape': 'x1'}, x1={'shape_by_conn': True}, x2={'shape_by_conn': True})
c1.add_expr('g1 = 2 * x1 - 5', g1={'copy_shape': 'x1'})
c1.add_expr('g2 = -2 * x1 - 5', g2={'copy_shape': 'x1'})
c1.add_expr('g3 = x2 - 5', g3={'copy_shape': 'x2'})

c1.add_design_var('x1', lower=-10, upper=10)
c1.add_design_var('x2', lower=-10, upper=10)
c1.add_objective('y')
c1.add_constraint('g1', lower=0)
c1.add_constraint('g2', lower=0)
c1.add_constraint('g3', lower=0)

prob.model.add_subsystem('c1', c1, promotes=['*'])
prob.driver = om.ScipyOptimizeDriver()

The feasible regions due to each constraint are shaded in the figure below.  Note that there is no solution which satisfies all three constraints.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

x1s = np.linspace(-10, 10, 100)
x2s = np.linspace(-10, 10, 100)
xx1, xx2 = np.meshgrid(x1s, x2s)

prob.setup()
prob.set_val('x1', xx1)
prob.set_val('x2', xx2)

prob.run_model()

ys = prob.get_val('y')

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.contour(x1s, x2s, ys, levels=20)
ax.fill_between(np.linspace(-10, 10, 10), 5 * np.ones(10), 10 * np.ones(10), color='r', alpha=0.3)
ax.fill_between(np.linspace(2.5, 10, 10), -10 * np.ones(10), 10 * np.ones(10), color='g', alpha=0.3)
ax.fill_between(np.linspace(-10, -2.5, 10), -10 * np.ones(10), 10 * np.ones(10), color='b', alpha=0.3)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.grid()
plt.show()

If we perform an optimization with the driver, it will fail due to the incompatible constraints.

In [ ]:
# Call setup again so we can use a new size for x1 and x2
prob.setup()

prob.set_val('x1', 1.0)
prob.set_val('x2', 1.0)

prob.run_driver()
prob.list_driver_vars(cons_opts=['lower', 'upper'], objs_opts=['val']);

If we use the `find_feasible` method on `Problem`, we can find the solution which minimizes the L2 norm of the constraint violations.
This method will attempt to find the solution that minimizes the constraint violations. If it fails to converge to a solution, it will return as having failed.
If it converges but the solution is still infeasible (it found the solution which minimizes the constraint violation), it will also return as having failed.

In [ ]:
x1_opt = prob.get_val('x1').copy()
x2_opt = prob.get_val('x2').copy()

prob.set_val('x1', 1.0)
prob.set_val('x2', 1.0)

prob.find_feasible()
prob.list_driver_vars(desvar_opts=['lower', 'upper'], cons_opts=['lower', 'upper'], objs_opts=['val']);

The following plot highlights the difference between the solution at the end of the failed optimization run, versus the solution found which minimizes feasibilities.

In [ ]:
x1_min_infeas = prob.get_val('x1').copy()
x2_min_infeas = prob.get_val('x2').copy()

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.contour(x1s, x2s, ys, levels=20)
ax.fill_between(np.linspace(-10, 10, 10), 5 * np.ones(10), 10 * np.ones(10), color='r', alpha=0.3)
ax.fill_between(np.linspace(2.5, 10, 10), -10 * np.ones(10), 10 * np.ones(10), color='g', alpha=0.3)
ax.fill_between(np.linspace(-10, -2.5, 10), -10 * np.ones(10), 10 * np.ones(10), color='b', alpha=0.3)
ax.plot(x1_opt, x2_opt, 'bo', ms=10, label='Optimizer solution')
ax.plot(x1_min_infeas, x2_min_infeas, 'k*', ms=10, label='find_feasible result')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.grid()
ax.legend()
plt.show()

## Omitting one or more design variables from the feasibility search

Sometimes the user may be interested in finding a feasible (or least infeasible) solution at a given value of one or more design variables.
The `find_feasible` method supports an `exclude_desvars` keyword argument that allows the user to exclude one or more design variables from the feasibility search. This argument may be a glob pattern which can match one or more design variable names.

For instance, let's find the minimimum-infeasibility-solution such that $x_1 = 5$.

In [ ]:
prob.set_val('x1', 5.0)

prob.find_feasible(exclude_desvars=['x1'])
prob.list_driver_vars(desvar_opts=['lower', 'upper'], cons_opts=['lower', 'upper'], objs_opts=['val']);

In [ ]:
x1_min_infeas_x1_fixed = prob.get_val('x1').copy()
x2_min_infeas_x1_fixed = prob.get_val('x2').copy()

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.contour(x1s, x2s, ys, levels=20)
ax.fill_between(np.linspace(-10, 10, 10), 5 * np.ones(10), 10 * np.ones(10), color='r', alpha=0.3)
ax.fill_between(np.linspace(2.5, 10, 10), -10 * np.ones(10), 10 * np.ones(10), color='g', alpha=0.3)
ax.fill_between(np.linspace(-10, -2.5, 10), -10 * np.ones(10), 10 * np.ones(10), color='b', alpha=0.3)
ax.plot(x1_opt, x2_opt, 'bo', ms=10, label='Optimizer solution')
ax.plot(x1_min_infeas, x2_min_infeas, 'k*', ms=10, label='find_feasible result')
ax.plot(x1_min_infeas_x1_fixed, x2_min_infeas_x1_fixed, 'r*', ms=10, label='find_feasible result $x_1=5$')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.grid()
ax.legend()
plt.show()

## Starting an Optimization from a Feasible Point

The `find_feasible` method iterates on the OpenMDAO model.

If it completes successfully, the model will be in a feasible state and optimization can begin immediately.

The following problem can be challenging to optimize from the default initial condition of `[1, 1, 10, 1]`.

This problem is the cost minimization of a pressure vessel with the design variables being the thicknesses of the end-cap and cylindrical sections (`d1` and `d2`), the vessel radius (`r`), and the length of the cylindrical section (`L`).

This example comes from _Solving Engineering Optimization Problems with the Simple Constrained Particle Swarm Optimizer_, by Cagnina et al., 2008.
Note that the original problem assumes integer values for `d1` and `d2` but they are continuous for the purpose of this demonstration.

In [ ]:
p = om.Problem()

p.model.add_subsystem('vessel',
                      om.ExecComp(['f = 0.6224 * d1 * r * L + 1.7781 * d2 * r**2 + 3.1661 * d1**2 * L + 19.84 * d1**2 * r',
                                   'g1 = -d1 + 0.0193 * r',
                                   'g2 = -d2 + 0.00954 * r',
                                   'g3 = -pi * r**2 * L - (4/3) * pi * r**3 + 1296000.'], has_diag_partials=True),
                      promotes=['*'])

p.model.add_design_var('d1', lower=0.0625)
p.model.add_design_var('d2', lower=0.0625, upper = 99 * 0.0625)
p.model.add_design_var('r', lower=10)
p.model.add_design_var('L', lower=1., upper=200.)

p.model.add_constraint('g1', upper=0)
p.model.add_constraint('g2', upper=0)
p.model.add_constraint('g3', upper=0, ref=1_000_000.)

p.model.add_objective('f', ref=1_000.)

p.driver = om.ScipyOptimizeDriver()

p.setup();

Instead, we'll reset the design point to the initial naive value and use find_feasible to first try to find a solution that satisfies all constraints.

In [ ]:
p.set_val('r', 10.) # Note we start r within its bounds for find_feasible.

p.run_model()
find_feas_fail = p.find_feasible()

In [ ]:
assert not find_feas_fail

The feasible starting point is:

In [ ]:
p.list_driver_vars(desvar_opts=['lower', 'upper'], cons_opts=['lower', 'upper'], objs_opts=[]);

Now the problem is in a state such that the problem is feasible, run the driver.

In [ ]:
opt_fail = p.run_driver()

p.list_driver_vars(desvar_opts=['lower', 'upper'], cons_opts=['lower', 'upper'], objs_opts=[]);